In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import feature
import scipy.misc as sm


# Generate noisy image of a square
root = '/home/lioruzan/pixel-cnn/data'
imname = 'P976-Fg001-R-C01-R02-D04082013-T145409-LR924 _012_mask.jpg'

img = sm.imread(os.path.join(root,imname))
kernel = np.ones((5,5),np.uint8)
im = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

# Compute the Canny filter
edges = feature.canny(im, sigma=3)
print(edges.shape)

(5412, 7216)


In [9]:
mask = 255-patch
yy, xx = mask.nonzero()







def generate_samples_mask(sess, X, h, pred, mask, init, conf, suff):
    print "Generating Sample Images..."
    n_row, n_col = 3, 9

    samples = np.zeros((n_row*n_col, conf.img_height, conf.img_width, conf.channel), dtype=np.float32)
    samples += init
    labels = one_hot(np.arange(conf.num_classes), conf.num_classes)

    for i in xrange(conf.img_height):
        for j in xrange(conf.img_width):
            data_dict = {X:samples}
            if conf.conditional is True:
                data_dict[h] = labels
            next_sample = sess.run(pred, feed_dict=data_dict)
            samples[:, i, j, :] = next_sample[:, i, j, :]*(1-mask[:,i,j,:]) + samples[:,i,j,:]*(mask[:,i,j,:])

    save_images(samples, n_row, n_col, conf, suff)







# sample from the model
new_x_gen = []
for i in range(args.nr_gpu):
    with tf.device('/gpu:%d' % i):
        gen_par = model(xs[i], h_sample[i], ema=ema, dropout_p=0, **model_opt)
        new_x_gen.append(nn.sample_from_discretized_mix_logistic(gen_par, args.nr_logistic_mix))
def sample_from_model(sess):
    x_gen = [samples[i] for i in range(args.nr_gpu)]
    for yi in range(obs_shape[0]):
        for xi in range(obs_shape[1]):
            new_x_gen_np = sess.run(new_x_gen, {xs[i]: x_gen[i] for i in range(args.nr_gpu)})
            for i in range(args.nr_gpu):
                x_gen[i][:,yi,xi,:] = new_x_gen_np[i][:,yi,xi,:]*(1-masks[:,yi,xi,:])+x_gen[i][:,yi,xi,:]*masks[:,yi,xi,:]
    return np.concatenate(x_gen, axis=0)



# train for one epoch
train_losses = []
for d in tqdm(train_data):
    feed_dict = make_feed_dict(d)
    # forward/backward/update model on each gpu
    lr *= args.lr_decay
    feed_dict.update({ tf_lr: lr })
    l,_ = sess.run([bits_per_dim, optimizer], feed_dict)
    train_losses.append(l)
train_loss_gen = np.mean(train_losses)

# compute likelihood over test data
test_losses = []
for d in test_data:
    feed_dict = make_feed_dict(d)
    l = sess.run(bits_per_dim_test, feed_dict)
    test_losses.append(l)
test_loss_gen = np.mean(test_losses)
test_bpd.append(test_loss_gen)

# log progress to console
print("Iteration %d, time = %ds, train bits_per_dim = %.4f, test bits_per_dim = %.4f" % (epoch, time.time()-begin, train_loss_gen, test_loss_gen))
sys.stdout.flush()

if epoch % args.save_interval == 0:

    
    for d in test_data:
        feed_dict = make_feed_dict(d)
        l = sess.run(bits_per_dim_test, feed_dict)
        test_losses.append(l)
    
    # generate samples from the model
    sample_x = sample_from_model(sess)
    img_tile = plotting.img_tile(sample_x[:int(np.floor(np.sqrt(args.batch_size*args.nr_gpu))**2)], aspect_ratio=1.0, border_color=1.0, stretch=True)
    img = plotting.plot_img(img_tile, title=args.data_set + ' samples')
    plotting.plt.savefig(os.path.join(args.save_dir,'%s_sample%d.png' % (args.data_set, epoch)))
    plotting.plt.close('all')



(array([  0,   0,   0, ..., 383, 383, 383]), array([359, 360, 361, ..., 381, 382, 383]))


In [4]:
import cv2
 
t=384
x,y = np.nonzero(edges)

for c in range(20):
    c = np.random.randint(len(x))
    i, j = x[c], y[c]
    while i<t/2 or i>=edges.shape[0]-t/2 or j<t/2 or j>=edges.shape[1]-t/2:
        c = np.random.randint(len(x))
        i,j = x[c], y[c]
    patch = im[i-t/2:i+t/2,j-t/2:j+t/2]
    sm.imshow(patch)

/home/lioruzan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
import matplotlib
import numpy as np
import scipy.misc as sm
import matplotlib.pyplot as plt

im = sm.face()
mask = np.ones_like(im)
print(im.shape)


def generate_samples_mask(sess, X, h, pred, mask, init, conf, suff):
    n_row, n_col = 3, 9
    samples = np.zeros((n_row*n_col, conf.img_height, conf.img_width, conf.channel), dtype=np.float32)
    samples += init
    labels = one_hot(np.arange(conf.num_classes), conf.num_classes)

    for i in xrange(conf.img_height):
        for j in xrange(conf.img_width):
            for k in xrange(conf.channel):
                data_dict = {X:samples}
                if conf.conditional is True:
                    data_dict[h] = labels
                next_sample = sess.run(pred, feed_dict=data_dict)
                if conf.data == "mnist" or conf.data == 'letters':
                    next_sample = binarize(next_sample)
                samples[:, i, j, k] = next_sample[:, i, j, k]*mask[i,j,k] + init[i,j,k]*(1-mask[i,j,k])